In [4]:
import pandas as pd
import numpy as np
import os
import json
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


import altair as alt
import hdbscan

import umap

from  itertools import *



In [7]:
datasets = [
    'boston', 
    'breastcancercoimbra',
    'breastcancerwisconsinprognostic',
    'covertype',
    'dermatology',
    'drybean',
    'echocardiogram',
    'ecoli',
    'extyaleb',
    'glassidentification',
    'heartdisease',
    'hepatitis',
    'housing', 'iris', 'mnist64', 'olive', 'weather', 'wine', 'world12d']

In [8]:
datasets_size = {
"boston": 70,
"breastcancercoimbra": 70,
"breastcancerwisconsinprognostic": 70,
"covertype": 65,
"dermatology": 70, #XX
"drybean": 65,
"echocardiogram": 70,
"ecoli": 70,
"extyaleb": 70,
"glassidentification": 70,
"heartdisease": 70,
"hepatitis": 65,
"housing": 65,
"iris": 70,
"mnist64": 70,
"olive": 70,  ##X
"weather": 70, 
"wine": 70, 
"world12d": 70
}

In [13]:
sm

NameError: name 'sm' is not defined

In [19]:
m = pd.read_json('./metric_all.json', orient='index')
s = pd.read_json('./scag_all.json', orient='index')
sm = pd.merge(s, m, left_index=True, right_index=True)

In [20]:
median = {}
for col in sm.columns:
    median[col] = sm[col].median()

for i, row in sm.iterrows():
    for col in row.index:
        if row[col] > median[col]:
            sm.loc[i, col] = True
        else:
            sm.loc[i, col] = False

s_bool = s

for i, row in s_bool.iterrows():
    for col in row.index:
        if row[col] > median[col]:
            s_bool.loc[i, col] = True
        else:
            s_bool.loc[i, col] = False
m_bool = m

for i, row in m_bool.iterrows():
    for col in row.index:
        if row[col] > median[col]:
            m_bool.loc[i, col] = True
        else:
            m_bool.loc[i, col] = False


In [ ]:
sm.drop(['stringy', 'DTM_KL1','DTM_KL001','Spearman', 'Continuity', 'Cohesiveness'], axis=1, inplace=True)
dataset = [True, False]

printList = list(product(dataset, repeat = len(sm.columns)))

x = {}
for i in printList:
    x[i] = []

for i in sm.columns:
    print(i, end=',')
print('mdp')

for i, row in sm.iterrows():
    x[tuple(row.values)].append(row.name)
for i in x:
    for ii in i:
        print(ii, end=',')
    for ii in x[i]:
        print(ii, end='/')
    print(',', end='')
    print(len(x[i]))

In [21]:
s_bool.drop(['stringy', 'monotonic'], axis=1, inplace=True)

dataset = [True, False]

printList = list(product(dataset, repeat = len(s_bool.columns)))

x = {}
for i in printList:
    x[i] = []

for i in s_bool.columns:
    print(i, end=',')
print('mdp')

outlying,convex,skinny,skewed,clumpy,striated,sparse,mdp


In [ ]:
for i, row in s_bool.iterrows():
    x[tuple(row.values)].append(row.name)
for i in x:
    for ii in i:
        print(ii, end=',')
    for ii in x[i]:
        print(ii, end='/')
    print(',', end='')
    print(len(x[i]))

In [198]:
m_bool.drop(['DTM_KL1','DTM_KL001','Spearman', 'Continuity', 'Cohesiveness'], axis=1, inplace=True)

dataset = [True, False]


printList = list(product(dataset, repeat = len(m_bool.columns)))

x = {}
for i in printList:
    x[i] = []

for i in m_bool.columns:
    print(i, end=',')
print('mdp')

RMSE, DTM_KL01, Sammon, Trustworthiness, Steadiness, mdp


In [30]:
# df_m = pd.read_csv('./metric_bin.csv', index_col=[0, 1, 2, 3, 4],keep_default_na=False)
df_s = pd.read_csv('./scag_bin.csv', index_col=[0, 1, 2, 3, 4, 5, 6],keep_default_na=False)
# df_sm = pd.read_csv('./all_bin.csv', index_col=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],keep_default_na=False)

In [10]:
binned_result_m = {}
for i , row in df_m.sort_values(by=['cnt'],ascending=False).iterrows():
    mdps = row.mdp.split('/') 
    charts = []
    for mdp in mdps:
        if (len(mdp.split('_')) < 2):
            continue
        dataset = mdp.split('_')[0]
        numProj = mdp.split('_')[1]
        xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
        xy.columns = ['x', 'y']
        label = pd.read_json(f'./ld/{dataset}/label.json')
        label.columns = ['label']
        data = pd.merge(xy, label, left_index=True, right_index=True)
        with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                meta = json.load(f)
        
        c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'{dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
        charts.append(c)
    title0 = "1:"
    title1 = "0:"
    for idx in range(len(i)):
        sc = df_m.index.names[idx]
        x = i[idx]
        if (x):
            title0 += sc + ', '
        else:
            title1 += sc + ', '
    title = title0 + ' / ' + title1
    num_per_row = 10
    x = []
    for ii in range(int(len(mdps) / num_per_row)):
            x.append(alt.hconcat(*charts[ii*num_per_row:(ii+1)*num_per_row], spacing=3))
    xx = alt.vconcat(*x).properties(title=alt.TitleParams(text=title, anchor='middle', color='darkblue', fontSize=20), spacing=3)
    binned_result_m[(tuple(i))] = xx

In [ ]:
idxx = 10
list(binned_result_m.values())[idxx]

In [31]:
idxx = 0
list(binned_result_s.values())[idxx]

IndexError: list index out of range

In [40]:
df_s.sort_values(by=['cnt'], ascending=False).cnt.values

array([143, 139,  47,  43,  36,  34,  33,  33,  32,  31,  27,  27,  26,
        26,  23,  21,  19,  19,  18,  18,  18,  17,  16,  14,  14,  14,
        14,  13,  13,  12,  12,  12,  11,  11,  11,  11,  11,  10,  10,
         9,   9,   9,   9,   9,   8,   8,   8,   8,   8,   8,   8,   7,
         6,   6,   6,   6,   5,   5,   5,   5,   5,   5,   5,   5,   5,
         5,   4,   4,   4,   4,   4,   4,   4,   4,   4,   3,   3,   3,
         3,   3,   3,   2,   2,   2,   2,   2,   2,   2,   2,   2,   2,
         1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,   1,
         1,   1,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [ ]:
binned_result_s = {}
for i , row in df_s.sort_values(by=['cnt'],ascending=False).iterrows():
    mdps = row.mdp.split('/') 
    charts = []
    for mdp in mdps:
        if (len(mdp.split('_')) < 2):
            continue
        dataset = mdp.split('_')[0]
        numProj = mdp.split('_')[1]
        xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
        xy.columns = ['x', 'y']
        label = pd.read_json(f'./ld/{dataset}/label.json')
        label.columns = ['label']
        data = pd.merge(xy, label, left_index=True, right_index=True)
        with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                meta = json.load(f)
        
        c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'{dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
        charts.append(c)
    title0 = "1:"
    title1 = "0:"
    for idx in range(len(i)):
        sc = df_s.index.names[idx]
        x = i[idx]
        if (x):
            title0 += sc + ', '
        else:
            title1 += sc + ', '
    title = title0 + ' / ' + title1
    num_per_row = 10
    x = []
    for ii in range(int(len(mdps) / num_per_row)):
            x.append(alt.hconcat(*charts[ii*num_per_row:(ii+1)*num_per_row], spacing=3))
    xx = alt.vconcat(*x).properties(title=alt.TitleParams(text=title, anchor='middle', color='darkblue', fontSize=20), spacing=3)
    binned_result_s[(tuple(i))] = xx

In [ ]:
binned_result_sm = {}
for i , row in df_sm.sort_values(by=['cnt'],ascending=False).iterrows():
    mdps = row.mdp.split('/') 
    charts = []
    for mdp in mdps:
        if (len(mdp.split('_')) < 2):
            continue
        dataset = mdp.split('_')[0]
        numProj = mdp.split('_')[1]
        xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
        xy.columns = ['x', 'y']
        label = pd.read_json(f'./ld/{dataset}/label.json')
        label.columns = ['label']
        data = pd.merge(xy, label, left_index=True, right_index=True)
        with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                meta = json.load(f)
        
        c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'{dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
        charts.append(c)
    title0 = "1:"
    title1 = "0:"
    for idx in range(len(i)):
        sc = df_sm.index.names[idx]
        x = i[idx]
        if (x):
            title0 += sc + ', '
        else:
            title1 += sc + ', '
    title = title0 + ' / ' + title1
    num_per_row = 10
    x = []
    for ii in range(int(len(mdps) / num_per_row)):
            x.append(alt.hconcat(*charts[ii*num_per_row:(ii+1)*num_per_row], spacing=3))
    xx = alt.vconcat(*x).properties(title=alt.TitleParams(text=title, anchor='middle', color='darkblue', fontSize=20), spacing=3)
    binned_result_sm[(tuple(i))] = xx

In [1]:
list(binned_result_s.values())[10]

NameError: name 'binned_result_s' is not defined

In [ ]:
for i, row in m_bool.iterrows():
    x[tuple(row.values)].append(row.name)
res = 0
for i in x:
    for ii in i:
        print(ii, end=',')
    for ii in x[i]:
        print(ii, end='/')
    print(',', end='')
    print(len(x[i]))

In [46]:
scagnostics = ['outlying', 'convex', 'skinny', 'stringy', 'monotonic', 'skewed',
       'clumpy', 'striated', 'sparse']
# scagnostics = ['sparse']
scags = {}
num_per_scag = 100
num_per_row = 10
for scag in scagnostics:
       high = s.sort_values(by=[scag], ascending=False).index[:num_per_scag]
       charts_high = []
       for x in high:
              dataset = x.split('_')[0]
              numProj = x.split('_')[1]
              xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
              xy.columns = ['x', 'y']
              label = pd.read_json(f'./ld/{dataset}/label.json')
              label.columns = ['label']
              data = pd.merge(xy, label, left_index=True, right_index=True)
              with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                     meta = json.load(f)
              c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'({round(s.loc[x][scag], 2)}) {dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
              charts_high.append(c)
       x = []
       for i in range(int(num_per_scag / num_per_row)):
              x.append(alt.hconcat(*charts_high[i*num_per_row:(i+1)*num_per_row], spacing=3))
       
       alt_high = alt.vconcat(*x).properties(title=alt.TitleParams(text=scag+" High 100", anchor='middle', color='darkblue', fontSize=20), spacing=3)
       
       low = s.sort_values(by=[scag], ascending=True).index[:num_per_scag]
       charts_low = []
       for x in low:
              dataset = x.split('_')[0]
              numProj = x.split('_')[1]
              xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
              xy.columns = ['x', 'y']
              label = pd.read_json(f'./ld/{dataset}/label.json')
              label.columns = ['label']
              data = pd.merge(xy, label, left_index=True, right_index=True)
              with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
                     meta = json.load(f)
              c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_circle(opacity=.5, size=20).encode(
                     x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
                     color=alt.Color('label:N', legend=None)
              ).properties(width=150, height=150, title=alt.TitleParams(text=f'({round(s.loc[x][scag], 2)}) {dataset}-{numProj}, {meta["method"]}', limit=150, fontSize=10))
              charts_low.append(c)
       x = []
       for i in range(int(num_per_scag / num_per_row)):
              x.append(alt.hconcat(*charts_low[i*num_per_row:(i+1)*num_per_row], spacing=3))
       
       alt_low = alt.vconcat(*x).properties(title=alt.TitleParams(text=scag+" low 100", anchor='middle', color='darkblue', fontSize=20), spacing=3)
       scags[scag] = (alt_high & alt_low)


In [ ]:
for s in scagnostics:
    print(f'<li><a href="./metric_viewer/scagpage/scag_{s}.html"><span style="color:#888">{s}</span></a></li>')

In [47]:
# scagnostics = ['outlying', 'convex', 'skinny', 'stringy', 'monotonic', 'skewed',
#        'clumpy', 'striated'. 'sparse']
idx = 7
for i in range(len(scagnostics)):
    scags[scagnostics[i]].save(f'./scag_{scagnostics[i]}.html')
# scags[scagnostics[0]]

In [99]:
charts = []
for x in a:
    dataset = x.split('_')[0]
    numProj = x.split('_')[1]
    xy = pd.read_json(f'./ld/{dataset}/ld_{str(numProj)}.json')
    xy.columns = ['x', 'y']
    label = pd.read_json(f'./ld/{dataset}/label.json')
    label.columns = ['label']
    data = pd.merge(xy, label, left_index=True, right_index=True)
    with open(f'./ld/{dataset}/metadata_{str(numProj)}.json') as f:
        meta = json.load(f)
    c = alt.Chart(pd.merge(xy, label, left_index=True, right_index=True)).mark_point().encode(
        x=alt.X('x', title=None, axis=alt.Axis(ticks=False, labels=False)),
        y=alt.Y('y', title=None, axis=alt.Axis(ticks=False, labels=False)),
        color=alt.Color('label:N', legend=None)
    ).properties(width=150, height=150, title=f'{dataset}-{numProj}, {meta["method"]}')
    charts.append(c)
    

In [ ]:

x = []
for i in range(int(30 / 5)):
    x.append(alt.hconcat(*charts[i*5:(i+1)*5]))
    
alt.vconcat(*x).properties(title=alt.TitleParams(text=, anchor='middle'))

In [3]:
metric_type = [
    'DTM_KL1',
    'DTM_KL01',
    'DTM_KL001',
    'RMSE',
    'Sammon',
    'Spearman',
    'Trustworthiness',
    'Continuity',
    'Steadiness',
    'Cohesiveness'
    ]

In [4]:
normalized_data = {}
scaled_data = {}
raw = {}

for dataset in datasets:
    boston = pd.read_csv(f'./metric_raw/{dataset}_metrics.csv', index_col='num')
    raw[dataset] = boston

    method = boston["method"]
    boston.drop(['method'], axis=1, inplace=True)


    scaled  = StandardScaler().fit_transform(boston)

    normalized = preprocessing.normalize(boston, axis=0)
    np_normalized = np.array(normalized)
    
    boston["method"] = method
    kmeans_result = KMeans(n_clusters=10).fit(np_normalized).predict(np_normalized)
    hdbscan_result = hdbscan.HDBSCAN(min_cluster_size=3).fit_predict(np_normalized)
    normalized_data[dataset] = np_normalized
    scaled_data[dataset] = scaled
    boston["cluster_kmeans"] = kmeans_result
    boston["cluster_hdbscan"] = hdbscan_result
    cluster_kmeans = {}
    cluster_hdbscan = {}
    for idx, data in boston.groupby('cluster_kmeans'):
        cluster_kmeans[idx] = data.index.tolist()

    for idx, data in boston.groupby('cluster_hdbscan'):
        cluster_hdbscan[idx] = data.index.tolist()

    # with open(f'hdbscan/clustering_{dataset}.json', 'w') as f:
    #     json.dump(cluster_hdbscan, f)

    # with open(f'kmeans/clustering_{dataset}.json', 'w') as f:
    #     json.dump(cluster_kmeans, f)

    # boston.to_json(f'./metric/{dataset}_metrics.json', orient='index')
    

In [82]:
reducer = umap.UMAP(n_neighbors=3, min_dist=0.1, n_components=2)

for dataset in datasets[:1]:
    res_n = reducer.fit_transform(normalized_data[dataset])
    res_s = reducer.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    charts = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title=dataset+' standard', width=200, height=200)


for dataset in datasets[1:]:
    res_n = reducer.fit_transform(normalized_data[dataset])
    res_s = reducer.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title=dataset+' standard', width=200, height=200)

    charts &= res_chart

In [ ]:
charts.title = 'UMAP(n_neighbors = 3, min_dist = 0.1, n_components=2)'
charts

In [5]:
from sklearn.manifold import TSNE

In [ ]:
chartss={}
for dataset in datasets[1:]:
    perplexity = 5

    tsne = TSNE(n_components=2, perplexity=perplexity)

    res_n = tsne.fit_transform(normalized_data[dataset])
    res_s = tsne.fit_transform(scaled_data[dataset])

    res_n = pd.DataFrame(res_n)
    res_s = pd.DataFrame(res_s)

    res_n.columns = ['x', 'y']
    res_s.columns = ['x', 'y']
    res_n.index.name = 'num'
    res_s.index.name = 'num'

    raw_d = raw[dataset]
    res_n = raw_d.merge(res_n, left_index=True, right_index=True)
    res_s = raw_d.merge(res_s, left_index=True, right_index=True)

    charts = alt.Chart(res_n).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
    ).properties(title='n /  perplexity : '+ str(perplexity), width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
        x='x',
        y='y',
        color='method',
        ).properties(title='s /perplexity : '+ str(perplexity), width=200, height=200)

    for perplexity in [10, 15, 20, 30, 40, 50]:
        tsne = TSNE(n_components=2, perplexity=perplexity)

        res_n = tsne.fit_transform(normalized_data[dataset])
        res_s = tsne.fit_transform(scaled_data[dataset])

        res_n = pd.DataFrame(res_n)
        res_s = pd.DataFrame(res_s)

        res_n.columns = ['x', 'y']
        res_s.columns = ['x', 'y']
        res_n.index.name = 'num'
        res_s.index.name = 'num'

        raw_d = raw[dataset]
        res_n = raw_d.merge(res_n, left_index=True, right_index=True)
        res_s = raw_d.merge(res_s, left_index=True, right_index=True)

        res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
            x='x',
            y='y',
            color='method',
        ).properties(title='perplexity : '+ str(perplexity), width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
            x='x',
            y='y',
            color='method',
            ).properties(title='perplexity : '+ str(perplexity), width=200, height=200)

        charts &= res_chart

    charts.title = 't-SNE /' + dataset
    chartss[dataset] = charts




# for dataset in datasets[1:]:
#     res_n = tsne.fit_transform(normalized_data[dataset])
#     res_s = tsne.fit_transform(scaled_data[dataset])

#     res_n = pd.DataFrame(res_n)
#     res_s = pd.DataFrame(res_s)

#     res_n.columns = ['x', 'y']
#     res_s.columns = ['x', 'y']
#     res_n.index.name = 'num'
#     res_s.index.name = 'num'

#     raw_d = raw[dataset]
#     res_n = raw_d.merge(res_n, left_index=True, right_index=True)
#     res_s = raw_d.merge(res_s, left_index=True, right_index=True)

#     res_chart = alt.Chart(res_n).mark_circle(size=10).encode(
#         x='x',
#         y='y',
#         color='method',
#     ).properties(title=dataset+' normalized', width=200, height=200) | alt.Chart(res_s).mark_circle(size=10).encode(
#         x='x',
#         y='y',
#         color='method',
#         ).properties(title=dataset+' standard', width=200, height=200)

#     charts &= res_chart